# Lab 2: Azure Logic Apps Integration with AI Foundry Agent

This notebook demonstrates how to integrate Azure Logic Apps workflows as tools within an AI Foundry agent.

## Learning Objectives
1. Configure Logic Apps client
2. Trigger Logic App workflows
3. Use workflow orchestration tools
4. Implement notification workflows

## Prerequisites
- Azure subscription
- Logic App with HTTP trigger deployed
- Python 3.10 or later
- Completed Lab 1 (recommended)

## Step 1: Setup and Imports

In [ ]:
import logging
import os
from dotenv import load_dotenv

from src.abstractions.logic_apps import (
    LogicAppConfig,
    LogicAppsClient,
    WorkflowOrchestrator,
    NotificationWorkflow,
    WorkflowStatus
)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv()

logger.info("Environment setup complete")

## Step 2: Configure Logic App Connection

Set up the connection to your Logic App workflow.

In [ ]:
# Configure Logic App
logic_app_config = LogicAppConfig(
    workflow_url=os.getenv(
        "LOGIC_APP_URL",
        "https://prod-123.eastus.logic.azure.com:443/workflows/..."
    ),
    subscription_id=os.getenv("AZURE_SUBSCRIPTION_ID"),
    resource_group=os.getenv("AZURE_RESOURCE_GROUP"),
    workflow_name=os.getenv("LOGIC_APP_NAME"),
    timeout=60
)

logger.info(f"Logic App configured: {logic_app_config.workflow_name}")

## Step 3: Create Logic Apps Client

In [ ]:
# Create the client
logic_apps_client = LogicAppsClient(logic_app_config)

logger.info("Logic Apps client initialized")

## Step 4: Trigger a Simple Workflow

Make a basic workflow trigger call.

In [ ]:
# Prepare workflow payload
workflow_payload = {
    "action": "process_order",
    "order_id": "ORD-12345",
    "customer": "John Doe",
    "items": [
        {"product": "Widget A", "quantity": 2},
        {"product": "Widget B", "quantity": 1}
    ]
}

# Trigger workflow
try:
    result = logic_apps_client.trigger_workflow(workflow_payload)
    logger.info(f"Workflow triggered: {result}")
    print("✅ Workflow triggered successfully!")
    print(f"Result: {result}")
except Exception as e:
    logger.error(f"Workflow trigger failed: {e}")
    print(f"❌ Error: {e}")

## Step 5: Async Workflow Triggering

Use async operations for better performance.

In [ ]:
import asyncio

async def trigger_async_workflow():
    """Demonstrate async workflow triggering."""
    payload = {
        "action": "send_report",
        "report_type": "monthly",
        "recipients": ["manager@company.com"]
    }
    
    try:
        result = await logic_apps_client.trigger_workflow_async(payload)
        logger.info(f"Async workflow result: {result}")
        print("✅ Async workflow triggered!")
        print(f"Result: {result}")
        return result
    except Exception as e:
        logger.error(f"Async trigger failed: {e}")
        print(f"❌ Error: {e}")
        return None

# Run async workflow
await trigger_async_workflow()

## Step 6: Use Workflow Orchestrator

The WorkflowOrchestrator provides a higher-level interface for common workflows.

In [ ]:
# Create orchestrator
orchestrator = WorkflowOrchestrator(logic_app_config)

# Execute approval workflow
try:
    result = orchestrator.execute_workflow(
        workflow_type="approval",
        data={
            "request_type": "budget_approval",
            "amount": 5000,
            "department": "Engineering",
            "requester": "alice@company.com"
        }
    )
    print("✅ Approval workflow executed!")
    print(f"Result: {result}")
except Exception as e:
    print(f"❌ Orchestration error: {e}")

## Step 7: Notification Workflow

Use the specialized notification workflow tool.

In [ ]:
# Create notification workflow
notifier = NotificationWorkflow(logic_app_config)

# Send high-priority notification
try:
    result = notifier.send_notification(
        recipient="admin@company.com",
        subject="System Alert: High CPU Usage",
        message="Server CPU usage has exceeded 90% for 10 minutes.",
        priority="high"
    )
    print("✅ Notification sent!")
    print(f"Result: {result}")
except Exception as e:
    print(f"❌ Notification error: {e}")

## Step 8: Complex Workflow Example

Demonstrate a multi-step business process.

In [ ]:
def process_customer_onboarding(customer_data: dict) -> dict:
    """Process customer onboarding through Logic App workflow."""
    orchestrator = WorkflowOrchestrator(logic_app_config)
    
    logger.info(f"Starting onboarding for {customer_data.get('name')}")
    
    try:
        # Trigger onboarding workflow
        result = orchestrator.execute_workflow(
            workflow_type="customer_onboarding",
            data=customer_data
        )
        
        logger.info("Onboarding workflow completed")
        return {
            "status": "success",
            "customer_id": customer_data.get("email"),
            "workflow_result": result
        }
    except Exception as e:
        logger.error(f"Onboarding failed: {e}")
        return {
            "status": "error",
            "error": str(e)
        }

# Test onboarding
customer = {
    "name": "Jane Smith",
    "email": "jane.smith@example.com",
    "company": "Acme Corp",
    "plan": "enterprise"
}

result = process_customer_onboarding(customer)
print(f"Onboarding result: {result}")

## Step 9: Workflow Status Enum

Use the WorkflowStatus enum for type-safe status handling.

In [ ]:
# Available workflow statuses
print("Available workflow statuses:")
for status in WorkflowStatus:
    print(f"  - {status.name}: {status.value}")

# Use in status checks
def check_workflow_status(status_value: str) -> None:
    """Check if a status value is valid."""
    try:
        status = WorkflowStatus(status_value)
        print(f"✅ Valid status: {status.name}")
    except ValueError:
        print(f"❌ Invalid status: {status_value}")

# Test status validation
check_workflow_status("Succeeded")
check_workflow_status("Running")
check_workflow_status("InvalidStatus")

## Step 10: Error Handling Best Practices

In [ ]:
import requests
from typing import Optional

def safe_workflow_trigger(
    client: LogicAppsClient,
    payload: dict,
    max_retries: int = 3
) -> Optional[dict]:
    """Safely trigger workflow with retry logic."""
    for attempt in range(max_retries):
        try:
            logger.info(f"Attempt {attempt + 1}/{max_retries}")
            result = client.trigger_workflow(payload)
            logger.info("Workflow triggered successfully")
            return result
        except requests.exceptions.Timeout:
            logger.warning(f"Timeout on attempt {attempt + 1}")
            if attempt == max_retries - 1:
                logger.error("Max retries reached")
                return None
        except Exception as e:
            logger.error(f"Unexpected error: {e}")
            return None
    return None

# Test with retry logic
result = safe_workflow_trigger(
    logic_apps_client,
    {"test": "data"},
    max_retries=2
)
print(f"Final result: {result}")

## 🎯 Copilot Tips for Logic Apps

### GitHub Copilot Chat Commands
- `/explain` - Understand Logic App workflow code
- `/fix` - Fix workflow configuration errors
- `/tests` - Generate workflow tests
- `/doc` - Document workflow integrations

### Useful Prompts
- "Create a Logic App workflow for email notifications"
- "Add error handling for workflow timeouts"
- "Generate tests for the NotificationWorkflow class"
- "Create a workflow that calls multiple services"

## 📚 Additional Resources

- [Azure Logic Apps Documentation](https://learn.microsoft.com/en-us/azure/logic-apps/)
- [Logic Apps Connectors](https://learn.microsoft.com/en-us/connectors/)
- [Workflow Definition Language](https://learn.microsoft.com/en-us/azure/logic-apps/logic-apps-workflow-definition-language)

## ✅ Summary

In this lab, you learned:
- ✅ Configuring Logic Apps clients
- ✅ Triggering workflows synchronously and asynchronously
- ✅ Using workflow orchestration tools
- ✅ Implementing notification workflows
- ✅ Error handling and retry logic

**Next**: Proceed to Lab 3 to integrate everything with an AI Foundry agent!